### Method 1

In [156]:
import sys
from qwikidata.sparql import return_sparql_query_results
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api
from SPARQLWrapper import SPARQLWrapper, JSON
import json
from tqdm import tqdm
import re
import requests as r
import pandas as pd
from collections import defaultdict
from libindic import inexactsearch
import pandas as pd
sys.path.append('../method2')
from People_translator import Translation_Api
translator = Translation_Api()

In [2]:
biography = {
    "image": "P18",
    "gender":"P21",
    "residence":"P551",
    "birth_place":"P19",
    "birth_date":"P569",
    "profession": "P106",
    "notable_works": "P800",
    "education": "P69",
    "positions":"P39",
    "awards": "P166",
    "spouse": "P26",
    "nationality": "P27",
}

translation = {
    "name":"नाम",
    "description":"विवरण",
    "image": "चित्र",
    "gender":"लिंग",
    "residence":"निवास",
    "birth_place":"जन्म स्थान",
    "birth_date":"जन्मतारीख",
    "profession": "व्यवसाय",
    "notable_works": "उल्लेखनीय कार्य",
    "education": "शिक्षा",
    "positions": "पद",
    "awards": "पुरस्कार",
    "spouse": "पति या पत्नी",
    "other_available_information":"अन्य उपलब्ध जानकारी",
    "main_info": "मुख्य जानकारी",
    "nationality": "राष्ट्रीयता"
}

to_transliteration = ['name','birth_place','spouse','notable_works']

In [3]:
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

endpoint_url = "https://query.wikidata.org/sparql"

In [4]:
#calls qwikidata get entity function for an entity id
def getEntityInfo(eid):
    return get_entity_dict_from_api(eid)

#extract the name of the entity or the property value in native language
def extractName(info):
    return info.get('labels', {}).get('en', {}).get('value', "")

#extract the description of the entity or the property value in native language
def extractDescription(info):
    return info.get('descriptions', {}).get('en', {}).get('value', "")
    
def method1_infobox(wd, bio, trans):
    result = {}
    #get entity from api
    entity_info = getEntityInfo(wd)
    result['name'] = translator.get_transliteration(extractName(entity_info))
    result['description'] = translator.get_translation(extractDescription(entity_info))
    #explicitly query using sparql to get main biography data
    for entity, wdt in bio.items():
        spqrqlq = f"SELECT ?entity ?entityLabel ?entityDescription WHERE {{ wd:{wd} wdt:{wdt} ?entity; SERVICE wikibase:label {{ bd:serviceParam wikibase:language \"en\". }} }}"
        s, v = trans[entity] + ": ", ""
        res = get_results(endpoint_url, str(spqrqlq))
        for entities in res['results']['bindings']:
            value = entities.get('entityLabel').get('value', "")
            if value != '' and 'Q' not in value:
                v += value + ','
        if v != "":
            if entity in to_transliteration:
                result[entity] = translator.get_transliteration(v)
            else:
                result[entity] = translator.get_translation(v)
    return result

### Method2

In [5]:
import wptools
import json
from tqdm import tqdm
import re
import requests as r
from collections import defaultdict

In [6]:
def update(infobox , translator):
    infobox = defaultdict(str , infobox)
    updated_infobox = {}
    updated_infobox['name'] = translator.get_transliteration(infobox['name'])
    
    updated_infobox['image'] = infobox['image']
    
    updated_infobox['caption'] = translator.get_translation(infobox['caption'])
    
    updated_infobox['fullname'] = translator.get_transliteration(infobox['fullname'])
    updated_infobox['nickname'] = translator.get_transliteration(infobox['nickname'])
    
    infobox['birth_date'] = infobox['birth_date'].replace("df|","df").replace("|yes","yes")
    updated_infobox['residence'] = translator.get_transliteration(infobox['residence'])
    updated_infobox['birth_date'] = translator.get_translation(infobox['birth_date'])
    updated_infobox['birth_place'] = translator.get_transliteration(infobox['birth_place'])
    
    updated_infobox['death_date'] = translator.get_translation(infobox['death_date'])
    updated_infobox['death_place'] = translator.get_transliteration(infobox['death_place'])
    
    updated_infobox['country'] = translator.get_transliteration(infobox['country'])
    updated_infobox['nationality'] = translator.get_translation(infobox['nationality'])
    updated_infobox['occupation'] = translator.get_translation(infobox['occupation'])
    updated_infobox['profession'] = translator.get_translation(infobox['profession'])
    updated_infobox['positions'] = translator.get_translation(infobox['positions'])
    updated_infobox['heightm'] = infobox['heightm']
    updated_infobox['gender'] = translator.get_translation(infobox['gender'])
    
    updated_infobox['spouse'] = translator.get_transliteration(infobox['spouse'])
    updated_infobox['children'] = translator.get_transliteration(infobox['children'])
    updated_infobox['parents'] = translator.get_transliteration(infobox['parents'])
    updated_infobox['father'] = translator.get_transliteration(infobox['father'])
    updated_infobox['mother'] = translator.get_transliteration(infobox['mother'])
    
    updated_infobox['party'] = translator.get_translation(infobox['party'])
    updated_infobox['awards'] = translator.get_transliteration(infobox['awards'])
    updated_infobox['relations'] = translator.get_translation(infobox['relations'])
    updated_infobox['known_for'] = translator.get_translation(infobox['known_for'])
    updated_infobox['notable_works'] = translator.get_transliteration(infobox['notable_works'])
    
    updated_infobox['alma_mater'] = translator.get_translation(infobox['alma_mater'])
    updated_infobox['education'] = translator.get_translation(infobox['education'])
    updated_infobox = { key : val for key , val in updated_infobox.items() if val!=''}
    cur_len = len(updated_infobox)
    if len(updated_infobox) < 15:
        for key , val in infobox.items():
            if key in updated_infobox : continue
            updated_infobox[key] = translator.get_transliteration(val)
            if len(updated_infobox) == 15: break
    updated_infobox = { key : val for key , val in updated_infobox.items() if val!=''}
    return updated_infobox

In [7]:
def method2_infobox(name):
    page = wptools.page(name).get_parse()
    result = update(page.data['infobox'] , translator)
    return result

### Baseline

In [8]:
def getEntityInfo(eid):
    return get_entity_dict_from_api(eid)

#extract the name of the entity or the property value in native language
def extractNameBaseline(info):
    return info.get('labels', {}).get('hi', {}).get('value', "")

#extract the description of the entity or the property value in native language
def extractDescriptionBaseline(info):
    return info.get('descriptions', {}).get('hi', {}).get('value', "")
    
def baseline_infobox(wd, bio, trans):
    result = {}
    #get entity from api
    entity_info = getEntityInfo(wd)
    #print name and description
    result['name'] = extractNameBaseline(entity_info)
    result['description'] = extractDescriptionBaseline(entity_info)
    #explicitly query using sparql to get main biography data
    for entity, wdt in bio.items():
        spqrqlq = f"SELECT ?entity ?entityLabel ?entityDescription WHERE {{ wd:{wd} wdt:{wdt} ?entity; SERVICE wikibase:label {{ bd:serviceParam wikibase:language \"hi\". }} }}"
        s, v = trans[entity] + ": ", ""
        res = get_results(endpoint_url, str(spqrqlq))
        for entities in res['results']['bindings']:
            value = entities.get('entityLabel').get('value', "")
            if value != '' and 'Q' not in value:
                v += value + ','
        if v != "":
            result[entity] = v
    return result
            

In [78]:
with open('../data-collection/hindi_person_data.jsonl') as f:
    data = f.readlines()
    data = [ json.loads(ent) for ent in data]
wiki_people = []
for entry in data:
    try :
        wiki_people.append([entry['en_wikipedia_title'] , entry['hi_wikipedia_title'] , entry['wd_id']])
    except :
        pass

In [117]:
def get_score(translated_infobox , actual_infobox):
    inst = inexactsearch.InexactSearch()
    markings = defaultdict(list)
    for key in translated_infobox.keys():
        if key in actual_infobox.keys() and key!='image':
#             print(key)
#             print(translated_infobox[key] , actual_infobox[key]) 
            val = inst.compare(translated_infobox[key] , actual_infobox[key])
            if val > 0.70 :
                markings['C'].append(key)
            else : markings['S'].append(key)
    for key in actual_infobox.keys():
        if key not in translated_infobox.keys():
            markings['D'].append(key)
    if len(markings['C']) + len(markings['S']) != 0:
        precision = len(markings['C']) / (len(markings['C']) + len(markings['S']))
    else : precision = 0
    recall = len(markings['C']) / (len(markings['C']) + len(markings['S']) + len(markings['D']))
    print('Precisions :',precision)
    print('Recall :',recall)
    return precision , recall

In [172]:
people_good = [
    ['Narendra Modi', 'नरेन्द्र मोदी', 'Q1058'],
    ['Indira Gandhi', 'इन्दिरा गांधी', 'Q1149'],
    ['Leonard Cohen', 'लिओनार्ड कोहेन', 'Q1276'],
    ['Donald Trump' , 'डॉनल्ड ट्रम्प' , 'Q22686'],
    ['Amitabh Bachchan','अमिताभ बच्चन','Q9570'],
    ['Sachin Tendulkar','सचिन तेंदुलकर','Q9488'],
    ['Virat Kohli','विराट कोहली','Q213854'],
    ['Joe Biden','जो बाइडेन','Q6279'],
    ['Malala Yousafzai' ,'मलाला युसुफ़ज़ई','Q32732']
]

In [189]:
scores_m1 = []
scores_m2 = []
scores_base = []
final_people = []
names = []
for en_name , hi_name , qid in people_good[:1]:
    print(en_name , hi_name , qid)
    actual_infobox = translator.get_page(hi_name , language = 'hi').data['infobox']
    if actual_infobox == None: continue
    method1 = method1_infobox(qid, biography,translation)
    method2 = method2_infobox(en_name)
    baseline = baseline_infobox(qid,biography,translation)
#     break
    print("For ", en_name)
    print("For method1")
    scores_m1.append(get_score(method1 , actual_infobox))
    print("For method2")
    scores_m2.append(get_score(method2 , actual_infobox))
    print("For baseline")
    scores_base.append(get_score(baseline , actual_infobox))
    names.append(en_name)
    if (scores_base[-1][0] > 0.5 or scores_m1[-1][0] > 0.5 or scores_m2[-1][0] > 0.5):
        final_people.append([en_name,hi_name,qid])
        print(final_people[-1])
        #     break

    if len(final_people) == 15: break
print()

Narendra Modi नरेन्द्र मोदी Q1058


hi.wikipedia.org (parse) नरेन्द्र मोदी
hi.wikipedia.org (imageinfo) File:PM Modi Portrait(cropped).jpg
नरेन्द्र मोदी (hi) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:PM Modi P...
  infobox: <dict(26)> image, office, president, term_start, predec...
  iwlinks: <list(3)> https://commons.wikimedia.org/wiki/Category:N...
  pageid: 26262
  parsetree: <str(111383)> <root><template><title>Infobox officeho...
  requests: <list(2)> parse, imageinfo
  title: नरेन्द्र मोदी
  wikibase: Q1058
  wikidata_url: https://www.wikidata.org/wiki/Q1058
  wikitext: <str(90526)> {{Infobox officeholder|image    = PM Modi...
}
en.wikipedia.org (parse) Narendra Modi
en.wikipedia.org (imageinfo) File:Prime Minister, Shri Narendra M...
Narendra Modi (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Prime Min...
  infobox: <dict(46)> image, image_size, order, office, president,...
  iwlinks: <list(3)> https://commons.wikimedia.org/wiki/Category:N...
  pageid: 444222
  parsetre

For  Narendra Modi
For method1
Precisions : 0.0
Recall : 0.0
For method2
Precisions : 0.625
Recall : 0.2
For baseline
Precisions : 0.25
Recall : 0.04
['Narendra Modi', 'नरेन्द्र मोदी', 'Q1058']



In [190]:
method1

{'name': 'नरेन्द्र मोदी',
 'description': 'भारत के प्रधानमंत्री',
 'image': 'http://commons.wikimedia.org/wiki/Special:FilePath/PM%20Modi%202015.jpg,',
 'gender': 'पुरुष',
 'residence': '7, लोक कल्याण मार्ग,',
 'birth_place': 'वडनगर,',
 'birth_date': '1950-09-17T00:00:00Z,',
 'profession': 'राजनेता',
 'education': 'गुजरात विश्वविद्यालय, दिल्ली विश्वविद्यालय,',
 'positions': 'भारत के प्रधानमंत्री, भारत की एक विधान सभा के सदस्य, कार्मिक, लोक शिकायत और पेंशन मंत्रालय, गुजरात विधान सभा के सदस्य, गुजरात के मुख्यमंत्री, 16वीं लोकसभा के सदस्य, 17वीं लोकसभा के सदस्य,',
 'awards': 'अब्दुलअजीज अल सऊद का आदेश, सेंट एंड्रयू का आदेश, जायद का आदेश, सीएनएन-न्यूज18 इंडियन ऑफ द ईयर, अमीर अमानुल्लाह खान पुरस्कार, फिलिस्तीन राज्य के ग्रैंड कॉलर, निशन इज्जुद्दीन के प्रतिष्ठित शासन का आदेश,',
 'spouse': 'जशोदबेन नरेंद्रभाई मोदी,',
 'nationality': 'भारत'}

In [191]:
method2

{'image': 'Prime Minister, Shri Narendra Modi, in New Delhi on August 08, 2019 (cropped).jpg',
 'residence': '[[7, लोक कल्याण मार्ग]], [[न्यू डेल्ही]], [[डेल्ही]], इंडिया',
 'birth_date': '{{birth date and age|1950|9|17|df=|y}}',
 'birth_place': '[[वडनगर]], [[बॉम्बे स्टेट]], [[इंडिया]] <br />(प्रेज़ेंट-डे [[गुजरात]])',
 'spouse': '[[Jashodaben|Jashodaben Modi]] ( {{abbr|m.|married}} &nbsp;1968; estranged)',
 'party': '[[भारतीय जनता पार्टी]]',
 'awards': '[[नरेन्द्र मोदी#स्टेट होनोर|लिस्ट ऑफ स्टेट होनोर]]',
 'alma_mater': '[[दिल्ली विश्वविद्यालय]] ([[बैचलर ऑफ आर्ट्स। बीए]]) <बीआर/> [[गुजरात विश्वविद्यालय]] ([[मास्टर ऑफ आर्ट्स]] । एमए]])',
 'image_size': '250पक्स',
 'order': '14थ',
 'office': 'प्राइम मिनिस्टर ऑफ इंडिया',
 'president': '[[प्रणब मुख़र्जी]]<br/>[[राम नाथ कोविंद]]',
 '1blankname': 'विस प्रेसिडेंट',
 '1namedata': '[[मोहम्मद हामिद अंसारी]]<br/>[[वेंकैया नायडू]]',
 'term_start': '26 मे 2014'}

In [194]:
baseline

{'name': 'नरेन्द्र मोदी',
 'description': 'भारत के प्रधानमंत्री',
 'image': 'http://commons.wikimedia.org/wiki/Special:FilePath/PM%20Modi%202015.jpg,',
 'gender': 'पुरुष,',
 'residence': '७, लोक कल्याण मार्ग,',
 'birth_place': 'वड़नगर,',
 'birth_date': '1950-09-17T00:00:00Z,',
 'profession': 'राजनीतिज्ञ,',
 'education': 'गुजरात विश्वविद्यालय,दिल्ली विश्वविद्यालय,',
 'positions': 'भारत का प्रधानमन्त्री,विधानसभा सदस्य (भारत),जनशिकायत मंत्रालय, भारत सरकार,गुजरात विधान सभा के सदस्य,',
 'awards': 'सीएनएन-आईबीएन इंडियन ऑफ़ द इयर,',
 'spouse': 'जशोदाबेन नरेन्द्रभाई मोदी,',
 'nationality': 'भारत,'}

In [109]:
actual_infobox = translator.get_page('डेविड बोवी' , language = 'hi').data['infobox']
print(actual_infobox)

hi.wikipedia.org (parse) डेविड बोवी


{'Name': 'David Bowie', 'Img': 'Heathen Tour Bowie.jpg', 'Img_capt': 'Bowie during the [[Heathen Tour]] in 2002.', 'Background': 'solo_singer', 'Birth_name': 'David Robert Jones (1947-2016)', 'Born': '{{Birth date |1947|1|8|df|=|yes}} <br />[[Brixton]], London, England - 10 january [[2016]].', 'Years_active': '1964–[[2016]]', 'Occupation': 'Musician, actor, record producer, arranger', 'Voice_type': 'Baritone', 'Instrument': 'Guitar, saxophone, piano, keyboards, synthesizers, [[Mellotron]], [[harmonica]], [[Stylophone]], [[xylophone]], [[vibraphone]], [[Koto (musical instrument)|koto]], drums, percussion', 'Genre': 'Rock, [[glam rock]], [[pop rock]], [[art rock]], [[blue-eyed soul]], [[psychedelic rock]], [[protopunk]]', 'Associated_acts': '[[The Riot Squad]], [[Tin Machine]]', 'Label': '[[Deram Records|Deram]], [[RCA Records|RCA]], [[Rykodisc]], [[Virgin Records|Virgin]], [[EMI Records|EMI]], [[ISO Records|ISO]], [[Columbia Records|Columbia]], [[Bertelsmann Music Group|BMG]], [[Parloph

डेविड बोवी (hi) data
{
  infobox: <dict(14)> Name, Img, Img_capt, Background, Birth_name,...
  iwlinks: <list(4)> https://commons.wikimedia.org/wiki/Category:D...
  pageid: 219951
  parsetree: <str(126632)> <root><template><title>Use British Engl...
  requests: <list(1)> parse
  title: डेविड बोवी
  wikibase: Q5383
  wikidata_url: https://www.wikidata.org/wiki/Q5383
  wikitext: <str(103037)> {{Use British English|date=अगस्त 2010}}{...
}


In [101]:
print(scores_base)

[(0.0, 0.0), (0.25, 0.04), (0.25, 0.018867924528301886), (0.3333333333333333, 0.04), (0.5, 0.07142857142857142)]


In [121]:
final_people2 = final_people.copy()
scores_base2 = scores_base.copy()
scores_m1 = scores_m1.copy()
scores_m2 = scores_m2.copy()

In [139]:
final_people

[['Narendra Modi', 'नरेन्द्र मोदी', 'Q1058'],
 ['Indira Gandhi', 'इन्दिरा गांधी', 'Q1149'],
 ['Edgar Allan Poe', 'एडगर ऍलन पो', 'Q16867'],
 ['Avril Lavigne', 'एव्रिल लावीन', 'Q30449']]

In [146]:
scores_m2

[(0.38461538461538464, 0.08620689655172414),
 (0.625, 0.2),
 (0.5454545454545454, 0.11320754716981132),
 (0.3333333333333333, 0.12),
 (0.5, 0.42857142857142855),
 (0, 0.0),
 (0.1111111111111111, 0.1111111111111111),
 (0.1111111111111111, 0.09090909090909091),
 (0, 0.0),
 (0.36363636363636365, 0.26666666666666666),
 (0.3333333333333333, 0.21428571428571427),
 (0.36363636363636365, 0.08333333333333333),
 (0.2857142857142857, 0.18181818181818182),
 (0, 0.0),
 (0.25, 0.08333333333333333),
 (0.5, 0.1),
 (0.3333333333333333, 0.09090909090909091),
 (0.4, 0.4),
 (0.4166666666666667, 0.25),
 (0.5, 0.21428571428571427),
 (0.1111111111111111, 0.025)]

In [174]:
len(scores_base)

18

In [175]:
names

['George Washington',
 'Narendra Modi',
 'Indira Gandhi',
 'Kofi Annan',
 'Leonard Cohen',
 'David Bowie',
 'Hokusai',
 'Abu Nuwas',
 'Muhammad',
 'A. P. J. Abdul Kalam',
 'Salman Khan',
 'Richard Nixon',
 'Charles K. Kao',
 'Bruce Lee',
 'Edgar Allan Poe',
 'Xavi',
 'Bo Xilai',
 'Hermann Hesse']

In [176]:
scores_m2_prec = [ i[0] for i in scores_m2]
scores_m1_prec = [ i[0] for i in scores_m1]
scores_base_prec = [ i[0] for i in scores_base]
# names = [ i[0] for i in names]

In [179]:
data = list(zip(names[:21] , scores_m1_prec , scores_m2_prec , scores_base_prec))
data = [ i for i in data if sum(i[1:])]

In [185]:
def highlight_max(s): 
    is_max = s == s.max() 
    return ['background: lightgreen' if cell else '' for cell in is_max] 

In [188]:
df = pd.DataFrame(data , columns=['Person' , "Method-1 Score", "Method-2 Score","Baseline Score"]).set_index("Person")
# df.style.highlight_max(color = 'lightgreen', axis = 0)
# df.style.apply(highlight_max)
df

,Method-1 Score,Method-2 Score,Baseline Score
Person,,,
George Washington,0.000000,0.384615,0.000000
Narendra Modi,0.000000,0.625000,0.250000
Indira Gandhi,0.250000,0.545455,0.250000
Kofi Annan,0.250000,0.333333,0.333333
Leonard Cohen,0.333333,0.500000,0.500000
Hokusai,0.000000,0.111111,0.333333
Abu Nuwas,0.333333,0.111111,0.333333
A. P. J. Abdul Kalam,0.000000,0.363636,0.250000
Salman Khan,0.200000,0.333333,0.200000


In [163]:
pd.DataFrame(data , columns=['Person' , "Method-1 Score", "Method-2 Score","Baseline Score"]).set_index("Person")

,Method-1 Score,Method-2 Score,Baseline Score
Person,,,
George Washington,0.000000,0.384615,0.000000
George W. Bush,0.000000,0.625000,0.250000
Nicolaus Copernicus,0.250000,0.545455,0.250000
Andrei Tarkovsky,0.250000,0.333333,0.333333
Meryl Streep,0.333333,0.500000,0.500000
Narendra Modi,0.000000,0.000000,0.000000
Indira Gandhi,0.000000,0.111111,0.333333
Kofi Annan,0.333333,0.111111,0.333333
Leonard Cohen,0.000000,0.000000,0.000000
